# 3D detector evaluation

In [11]:
import os
import numpy as np
import glob
from bbox_utils import BoundingBox3D

# Convert the pickle results file into txt file:
First - convert the results.pkl file in the output directory to txt files using get_results.py

# Metrics calculation:

In [12]:
def calculate_metrics(tp, fp, fn, tn):
    EPSILON = 1e-10  # Avoid division by zero
    
    precision = tp / (tp + fp + EPSILON)
    recall = tp / (tp + fn + EPSILON)
    f1_score = 2 * (precision * recall) / (precision + recall + EPSILON)

    return precision, recall, f1_score

In [13]:
def calculate_iou(box1, box2):
    """
    Calculate IoU (Intersection over Union) between two bounding boxes.
    """
    x1, y1, z1, w1, h1 = box1[:5]
    x2, y2, z2, w2, h2 = box2[:5]

    intersect_x = max(0, min(x1 + w1 / 2, x2 + w2 / 2) - max(x1 - w1 / 2, x2 - w2 / 2))
    intersect_y = max(0, min(y1 + h1 / 2, y2 + h2 / 2) - max(y1 - h1 / 2, y2 - h2 / 2))

    intersection = intersect_x * intersect_y
    union = w1 * h1 + w2 * h2 - intersection

    iou = intersection / max(union, 1e-10)  # Avoid division by zero

    return iou

In [14]:
# def evaluate_detection(gt_boxes, dnn_boxes, iou_threshold=0.5, confidence_threshold=0.3):
#     tp = fp = fn = 0
#     matched_dnn_boxes = set()  # To keep track of which dnn boxes have been matched
#     boxes_to_discount = set()  # Due to low confidence score

#     # Evaluate True Positives (TP) and False Negatives (FN)
#     for gt_box in gt_boxes:
#         match_found = False
#         for idx, dnn_box in enumerate(dnn_boxes):  
#             if idx in matched_dnn_boxes:
#                 continue  # Skip boxes that are already matched

#             confidence = dnn_box[7]
#             if confidence < confidence_threshold:
#                 boxes_to_discount.add(idx)

#             if calculate_iou(gt_box, dnn_box) >= iou_threshold and confidence >= confidence_threshold:
#                 tp += 1
#                 match_found = True
#                 matched_dnn_boxes.add(idx)
#                 break  # Break after finding the first match

#         if not match_found:
#             fn += 1

#     # Evaluate False Positives (FP)
#     fp = len(dnn_boxes) - len(matched_dnn_boxes) - len(boxes_to_discount) # Total dnn_boxes minus those that matched
    
#     return tp, fp, fn

In [15]:
def evaluate_detection(gt_boxes, dnn_boxes, iou_threshold=0.3, confidence_threshold=0.3):
    tp = fp = fn = 0
    matched_dnn_boxes = set()  # To keep track of which dnn boxes have been matched
    boxes_to_discount = set()  # Due to low confidence score

    # Separate the dnn_boxes based on confidence threshold
    high_conf_dnn_boxes = [(idx, box) for idx, box in enumerate(dnn_boxes) if box[7] >= confidence_threshold]
    low_conf_dnn_boxes = set(idx for idx, box in enumerate(dnn_boxes) if box[7] < confidence_threshold)

    # Evaluate True Positives (TP) and False Negatives (FN)
    for gt_box in gt_boxes:
        match_found = False
        for idx, dnn_box in high_conf_dnn_boxes:
            if idx in matched_dnn_boxes:
                continue  # Skip boxes that are already matched

            if calculate_iou(gt_box, dnn_box) >= iou_threshold:
                tp += 1
                match_found = True
                matched_dnn_boxes.add(idx)
                break  # Break after finding the first match

        if not match_found:
            fn += 1

    # Evaluate False Positives (FP)
    fp = len(dnn_boxes) - len(matched_dnn_boxes) - len(low_conf_dnn_boxes) # Total dnn_boxes minus those that matched and those with low confidence

    return tp, fp, fn


# Visualization

In [16]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def saveHTML(pc, gt_boxes, detector_boxes, file_path, file_path_png, show_dont_save=True, margin=3, min_confidence=0.0):
    pc = pc.T
    data = []

    # Plot the point cloud
    data.append(go.Scatter3d(
        x=pc[0,:], y=pc[1,:], z=pc[2,:],
        mode="markers",
        marker=dict(size=2, color=pc[2,:], colorscale='Viridis', opacity=1)
    ))

    # Function to format bbox data text
    def format_bbox_text(x, y, z, confidence=None):
        text = f"X: {x:.2f}, Y: {y:.2f}, Z: {z:.2f}"
        if confidence is not None:
            text += f"<br>Confidence: {confidence*100:.2f}%"
        return text

    # Add Detector Bounding Boxes in Red with confidence scores
    for det in detector_boxes:
        confidence = det[7]
        if confidence >= min_confidence:
            bbox = BoundingBox3D(
                x=det[0], y=det[1], z=det[2],
                length=det[3], width=det[4], height=det[5],
                euler_angles=[0, 0, det[6]]
            )
            corners, triangle_vertices = bbox.p, bbox.triangle_vertices

            # Dynamic opacity based on confidence
            # opacity = 0.2 + 0.2 * confidence

            # Mesh for the bounding box
            data.append(go.Mesh3d(
                x=corners[:, 0], y=corners[:, 1], z=corners[:, 2],
                i=triangle_vertices[0], j=triangle_vertices[1], k=triangle_vertices[2],
                opacity=0.3, color='red',
                hovertext=format_bbox_text(det[0], det[1], det[2], confidence),
                hoverinfo='text'
            ))

    # Add Ground Truth Bounding Boxes in Green
    for gt in gt_boxes:
        bbox = BoundingBox3D(
            x=gt[0], y=gt[1], z=gt[2],
            length=gt[3], width=gt[4], height=gt[5],
            euler_angles=[0, 0, gt[6]]
        )
        corners, triangle_vertices = bbox.p, bbox.triangle_vertices
        center_x = (corners[:,0].max() + corners[:,0].min()) / 2
        center_y = (corners[:,1].max() + corners[:,1].min()) / 2
        center_z = (corners[:,2].max() + corners[:,2].min()) / 2
        
        # Mesh for the bounding box
        data.append(go.Mesh3d(
            x=corners[:, 0], y=corners[:, 1], z=corners[:, 2],
            i=triangle_vertices[0], j=triangle_vertices[1], k=triangle_vertices[2],
            opacity=0.3, color='rgb(0, 255, 0)', # Green for ground truth
            flatshading=True
        ))

        # Text for bbox details
        data.append(go.Scatter3d(
            x=[center_x], y=[center_y], z=[center_z],
            mode='text',
            text=[format_bbox_text(gt[0], gt[1], gt[2])],
            textposition='middle center',
            showlegend=False,
            marker=dict(color='rgb(255, 255, 255)')
        ))

    # Define the layout with specified aesthetics
    layout = go.Layout(
        scene=dict(
            xaxis=dict(nticks=40, range=[-20, 20], showbackground=True, backgroundcolor='rgb(30, 30, 30)', gridcolor='rgb(127, 127, 127)', zerolinecolor='rgb(127, 127, 127)'),
            yaxis=dict(nticks=40, range=[-20, 20], showbackground=True, backgroundcolor='rgb(30, 30, 30)', gridcolor='rgb(127, 127, 127)', zerolinecolor='rgb(127, 127, 127)'),
            zaxis=dict(nticks=20, range=[-1, 19], showbackground=True, backgroundcolor='rgb(30, 30, 30)', gridcolor='rgb(127, 127, 127)', zerolinecolor='rgb(127, 127, 127)'),
            xaxis_title="x (meters)",
            yaxis_title="y (meters)",
            zaxis_title="z (meters)"
        ),
        margin=dict(r=10, l=10, b=10, t=10),
        paper_bgcolor='rgb(30, 30, 30)',
        font=dict(family="Courier New, monospace", color='rgb(127, 127, 127)'),
        legend=dict(font=dict(family="Courier New, monospace", color='rgb(127, 127, 127)'))
    )
    
    fig = go.Figure(data=data, layout=layout)
    if show_dont_save == False:
        fig.write_html(file_path)
    else:
        fig.show()

    if show_dont_save == False:
        # Save as PNG
        fig.update_layout(scene_camera=dict(eye=dict(x=1, y=0, z=0)))
        fig.write_image(file_path_png)

# File processing

In [17]:
# modified cell to see if it works
def read_gt_file(fname):

    with open(fname, 'r') as file:
        lines = file.readlines()

    data = []

    for line in lines:
        # Split each line into a list of strings
        values = line.strip().split()

        # Convert numerical values to floats and keep the last element as a string
        values = [float(val) if i < len(values) - 1 else val for i, val in enumerate(values)]

        # Append the list of values to the data list
        data.append(values)
        
    return data

In [18]:
def read_detector_file(fname):

    with open(fname, 'r') as file:
        lines = file.readlines()

    data = []

    for line in lines:
        # Split each line into a list of strings
        values = line.strip().split()

        # Convert strings to floats
        values = [float(val) for val in values]

        # Append the list of values to the data list
        data.append(values)
        
    return data

In [19]:
# Specify the names of the directories:
def process_files(base_dir, evaluate_all=True, single_file=None):
    detector_path = os.path.join(base_dir, 'detector_labels')
    gt_path = os.path.join(base_dir, 'gt_test/labels')
    npy_path = os.path.join(base_dir, 'gt_test/npy')
    html_path = os.path.join(base_dir, 'gt_test/html')
    
    file_list = [os.path.join(detector_path, single_file)] if not evaluate_all else glob.glob(os.path.join(detector_path, '*.txt'))

    total_tp = total_fp = total_fn = 0

    for file_name in file_list:
        basename = os.path.splitext(os.path.basename(file_name))[0]
        gt_file = os.path.join(gt_path, f"{basename}.txt")
        npy_file = os.path.join(npy_path, f"{basename}.npy")

        detector_boxes = read_detector_file(file_name)
        gt_boxes = read_gt_file(gt_file)
        pc = np.load(npy_file)

        tp, fp, fn = evaluate_detection(gt_boxes, detector_boxes)
        total_tp += tp
        total_fp += fp
        total_fn += fn

        results_dir = os.path.join(html_path, f"{basename}_results")
        os.makedirs(results_dir, exist_ok=True)
        file_path_html = os.path.join(results_dir, f"{basename}.html")
        file_path_png = os.path.join(results_dir, f"{basename}.png")
        saveHTML(pc, gt_boxes, detector_boxes, file_path_html, file_path_png, show_dont_save=True) # Set to False to save the HTML and PNG files

    return total_tp, total_fp, total_fn

# Print evaluation metrics:

In [21]:
base_dir = '/home/tauproj1/Innoviz_Project/OpenPCDet-master_New_copy/3D_Object_Detector/Evaluation/pointpillar_enhanced2' # and remember to also change the file names and "show_dont_save" in the cell above if needed
total_tp, total_fp, total_fn = process_files(base_dir, evaluate_all=True)                                  # Evaluate all files
# total_tp, total_fp, total_fn = process_files(base_dir, evaluate_all=False, single_file='res_00001.txt')  # Evaluate a single file
precision, recall, f1_score = calculate_metrics(total_tp, total_fp, total_fn, 0)  # tn is not used
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1_score:.4f}")

Total True Positives: 0, Total False Positives: 0, Total False Negatives: 0
Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000
